/*
Covid 19 Data Exploration 

Skills used: Joins, CTE's, Temp Tables, Windows Functions, Aggregate Functions, Creating Views, Converting Data Types

*/

--Select the Data that we are going to be using

In [ ]:
SELECT Location, date, total_cases, new_cases, total_deaths, population
FROM `portofolioproject-421721.Covid_Vaccinations.Covid_deaths`
ORDER BY 1,2

--Looking at total cases vs total deaths
--SHows the likelihood of dyin if you contract covid your country

In [ ]:
SELECT Location, date, total_cases, total_deaths, (total_deaths/total_cases)*100 as DeathPercentage
FROM `portofolioproject-421721.Covid_Vaccinations.Covid_deaths`
WHERE location like '%United States%'
ORDER BY 1,2

--Looking at Total Cases vs Population
--SHows what percentage of population got covid

In [ ]:
SELECT Location, date, total_cases, population, (total_cases/population)*100 as DeathPercentage
FROM `portofolioproject-421721.Covid_Vaccinations.Covid_deaths`
--WHERE location like '%United States%'
ORDER BY 1,2

--Looking at Countries with highest infection rates compared to population

In [ ]:
SELECT Location, population, MAX(total_cases) as Highest_Infection_count,Max((total_cases/population))*100 as PercentPopulationInfected
FROM `portofolioproject-421721.Covid_Vaccinations.Covid_deaths`
--WHERE location like '%United States%'
GROUP BY location, population
ORDER BY PercentPopulationInfected desc

-- Showing the countries with the highest death counth per population


In [ ]:
SELECT Location, MAX(cast(total_deaths as int)) as TotalDeathCount
FROM `portofolioproject-421721.Covid_Vaccinations.Covid_deaths`
--WHERE location like '%United States%'
WHERE continent is not null
GROUP BY location
ORDER BY TotalDeathCount desc

-- Let's break things down by continent
-- Showing the continents with the highst death count

In [ ]:
SELECT continent, MAX(cast(total_deaths as int)) as TotalDeathCount
FROM `portofolioproject-421721.Covid_Vaccinations.Covid_deaths`
--WHERE location like '%United States%'
WHERE continent is not null
GROUP BY continent
ORDER BY TotalDeathCount desc

-- Global Numbers


In [ ]:
SELECT SUM(new_cases) as total_cases, SUM(cast(new_deaths as int)) as total_deaths, SUM(cast(new_deaths as int))/SUM(New_Cases)*100 as DeathPercentage
FROM `portofolioproject-421721.Covid_Vaccinations.Covid_deaths`
--Where location like '%states%'
WHERE continent is not null 
--Group By date
ORDER BY 1,2

-- Looking at total Population vs Vaccinations

In [ ]:
SELECT dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(Cast(vac.new_vaccinations as INT64)) OVER (Partition by dea.location ORDER BY dea.location,dea.date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
FROM `portofolioproject-421721.Covid_Vaccinations.Covid_deaths` dea
JOIN `portofolioproject-421721.Covid_Vaccinations.Covid_Vaccinations` vac
  ON dea.location = vac.location
  and dea.date = vac.date
WHERE dea.continent is not null
ORDER BY 2,3

-- USE CTE

In [ ]:
WITH Recursive
  PopvsVac AS (
SELECT dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(Cast(vac.new_vaccinations as INT64)) OVER (Partition by dea.location ORDER BY dea.location,dea.date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
FROM `portofolioproject-421721.Covid_Vaccinations.Covid_deaths` dea
JOIN `portofolioproject-421721.Covid_Vaccinations.Covid_Vaccinations` vac
  ON dea.location = vac.location
  and dea.date = vac.date
WHERE dea.continent is not null
--ORDER BY 2,3
)
SELECT *, (RollingPeopleVaccinated/Population)*100
FROM PopvsVac

-- TEMP TABLE #############

In [ ]:
CREATE TABLE IF NOT EXISTS portofolioproject-421721.PercentPopulationVaccinated(
Continent STRUCT<a ARRAY<STRING>,
Location STRUCT<a ARRAY<STRING>,
Date INT64,
Population INT64,
New_vaccinations STRUCT<a ARRAY<STRING>,
RollingPeopleVaccinated STRUCT<a ARRAY<STRING>)

INSERT INTO PercentPopulationVaccinated
SELECT dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(Cast(vac.new_vaccinations as INT64)) OVER (Partition by dea.location ORDER BY dea.location,dea.date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
FROM `portofolioproject-421721.Covid_Vaccinations.Covid_deaths` dea
JOIN `portofolioproject-421721.Covid_Vaccinations.Covid_Vaccinations` vac
  ON dea.location = vac.location
  and dea.date = vac.date
WHERE dea.continent is not null
--ORDER BY 2,3

SELECT *, (RollingPeopleVaccinated/Population)*100
FROM PercentPopulationVaccinated

-- Creating view to store data for later visualizations

In [ ]:
Create view PercentPopulationVaccinated as
SELECT dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(Cast(vac.new_vaccinations as INT64)) OVER (Partition by dea.location ORDER BY dea.location,dea.date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
FROM `portofolioproject-421721.Covid_Vaccinations.Covid_deaths` dea
JOIN `portofolioproject-421721.Covid_Vaccinations.Covid_Vaccinations` vac
  ON dea.location = vac.location
  and dea.date = vac.date
WHERE dea.continent is not null
--ORDER BY 2,3
